In [3]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

## 1. 对话缓冲内存

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

In [9]:
chat = ChatOpenAI(temperature=0)

memory = ConversationBufferMemory()
conversation = ConversationChain(llm=chat, memory=memory, verbose=True)

In [10]:
conversation.predict(input="Hi, my name is Eugene.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Eugene.
AI:

> Finished chain.


"Hello Eugene! It's nice to meet you. How can I assist you today?"

In [11]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Eugene.
AI: Hello Eugene! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'1+1 is equal to 2.'

In [13]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Eugene.
AI: Hello Eugene! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my name?
AI:

> Finished chain.


'Your name is Eugene.'

In [20]:
print(memory.buffer)

Human: Hi, my name is Eugene.
AI: Hello Eugene! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my name?
AI: Your name is Eugene.


In [21]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Eugene.\nAI: Hello Eugene! It's nice to meet you. How can I assist you today?\nHuman: What is 1+1?\nAI: 1+1 is equal to 2.\nHuman: What is my name?\nAI: Your name is Eugene."}

**demo**

In [18]:
mem1 = ConversationBufferMemory()

mem1.save_context({"input": "Hi"}, {"output": "What's up"})
print(mem1.buffer)

Human: Hi
AI: What's up


In [22]:
mem1.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

In [23]:
mem1.save_context({"input": "Not much, just hanging"}, {"output": "Cool"})

In [24]:
mem1.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

***SUMMARY***

1. LLMs are 'stateless'
2. Each transaction is independent
3. Chatbos appear to have memory by providing *the full conversation* as 'context'

But with OpenAI's **limit on the number of prompt tokens**, this approach doesn't seem to work when having a long conversation.

**Langchain provides serveral kinds of 'memory' to store and accumulate the conversation.**

## 2. 对话缓冲窗口内存

In [25]:
from langchain.memory import ConversationBufferWindowMemory

In [26]:
# k: num of last conversations to save in memory
window_memory = ConversationBufferWindowMemory(k=1)

**demo**

In [27]:
window_memory.save_context({"input": "Hi"}, {"output": "What's up"})
window_memory.save_context({"input": "Not much, just hanging"}, {"output": "Cool"})

In [30]:
# buffered conversations in 'window memory'
window_memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

**demo ends.**

In [31]:
window_memory = ConversationBufferWindowMemory(k=1)
conversation2 = ConversationChain(llm=chat, memory=window_memory, verbose=True)

In [32]:
conversation2.predict(input="Hi, my name is Eugene.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Eugene.
AI:

> Finished chain.


"Hello Eugene! It's nice to meet you. How can I assist you today?"

In [33]:
conversation2.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Eugene.
AI: Hello Eugene! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'1+1 is equal to 2.'

In [34]:
conversation2.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my name?
AI:

> Finished chain.


"I'm sorry, but I don't have access to personal information about individuals unless it has been shared with me in the course of our conversation."

In [35]:
window_memory.buffer

"Human: What is my name?\nAI: I'm sorry, but I don't have access to personal information about individuals unless it has been shared with me in the course of our conversation."

## 3. 对话Token缓冲区内存

In [36]:
from langchain.memory import ConversationTokenBufferMemory

In [52]:
# max_token_limit: num of last conversation tokens to save in memory
token_memory = ConversationTokenBufferMemory(llm=chat, max_token_limit=50)

In [53]:
token_memory.save_context({"input": "AI is what?"}, {"output": "Amazing"})
token_memory.save_context({"input": "Backpropagation is what?"}, {"output": "Beautiful!"})
token_memory.save_context({"input": "Chatbos are what?"}, {"output": "Charming!"})

In [54]:
token_memory.load_memory_variables({})

{'history': 'AI: Amazing\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbos are what?\nAI: Charming!'}

`max_token_limit`: ***num of last conversation tokens*** to save in memory.

e.g.

```
max_token_limit  -->  buffered conversation string
50  -->  AI: Amazing\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbos are what?\nAI: Charming!
30  -->  AI: Beautiful!\nHuman: Chatbos are what?\nAI: Charming!
20  ->>  AI: Charming!
```

## 4. 对话总结缓冲内存

***Use LLM to write summary of conversations so far and keep it in memory.***

In [55]:
 from langchain.memory import ConversationSummaryBufferMemory

In [56]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

In [77]:
summary_memory = ConversationSummaryBufferMemory(llm=chat, max_token_limit=100)

# 'create conversations'
summary_memory.save_context({"input": "Hello"}, {"output": "What's up"})
summary_memory.save_context({"input": "Not much, just hanging"}, {"output": "Cool"})
summary_memory.save_context({"input": "What is on the schedule today?"}, {"output": f"{schedule}"}) 

In [78]:
summary_memory.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings. The human mentions that they are not doing much. The AI informs the human about their schedule for the day, including a meeting with the product team, working on the LangChain project, and having lunch with a customer to discuss the latest in AI. The AI also reminds the human to bring their laptop to show a demo.'}

some examples when `max_token_limit` changes:

***max_token_limit = 400***   (when 400 tokens is big enough to contain all conversations)

Human: Hello\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool\nHuman: What is on the schedule today?\nAI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.

***max_token_limit = 100***

System: The human and AI exchange greetings. The human mentions that they are not doing much. The AI informs the human about their schedule for the day, including a meeting with the product team, working on the LangChain project, and having lunch with a customer to discuss the latest in AI. The AI also reminds the human to bring their laptop to show a demo.


In [80]:
conversation4 = ConversationChain(llm=chat, memory=summary_memory, verbose=True)

conversation4.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI exchange greetings. The human mentions that they are not doing much. The AI informs the human about their schedule for the day, including a meeting with the product team, working on the LangChain project, and having lunch with a customer to discuss the latest in AI. The AI also reminds the human to bring their laptop to show a demo.
Human: What would be a good demo to show?
AI:

> Finished chain.


'A good demo to show would be a live demonstration of the LangChain project. You can showcase the features and functionality of the language translation software, highlighting its accuracy and efficiency. Additionally, you could also demonstrate any recent updates or improvements made to the project. This would give the customer a firsthand experience of the capabilities of the AI technology and its potential benefits for their business.'

In [82]:
summary_memory.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings. The human mentions that they are not doing much. The AI informs the human about their schedule for the day, including a meeting with the product team, working on the LangChain project, and having lunch with a customer to discuss the latest in AI. The AI also reminds the human to bring their laptop to show a demo.\nHuman: What would be a good demo to show?\nAI: A good demo to show would be a live demonstration of the LangChain project. You can showcase the features and functionality of the language translation software, highlighting its accuracy and efficiency. Additionally, you could also demonstrate any recent updates or improvements made to the project. This would give the customer a firsthand experience of the capabilities of the AI technology and its potential benefits for their business.'}

## 5. 总结

***ConversationBufferMemory***

This memory allows for storing of messages and then extracts the messages in a variable.

***ConversationBufferWindowMemory***

This memory keeps a list of the interactions of the conversation over time. It only uses the last K interactions.

***ConversationTokenBufferMemory***

This memory keeps a buffer of recent interactions in memory, and uses token length rather than number of interactions to determine when to flush interactions.

***ConversationSummaryMemory***

This memory creates a summary of the conversation over time.

---

Langchain also provides other types of memory:

***Vector data memory***

stores text(from conversation or elsewhere) in a vector database and retrieves the most relevant blocks of text.

***Entity memory***

using a LLM, it remembers details about specific entities.

---

You can alse use multiple memories at one time. E.g., Conversation Memory + Entity Memory, to recall individuals

You can also store the conversation in a conventional database, such as KV store or SQL.
